In [1]:
data = """789
456
123
.0A"""

data_robot = """.^A
<v>"""


def parse(data):

    direct = {}
    for (y,l) in  enumerate(data.splitlines()):
        for (x,c) in enumerate(l):
            if c == '.':
                continue
            direct[(x,y)] = c
    return tuple((k,v) for k,v in direct.items())
keypad = parse(data)
rob = parse(data_robot)



In [2]:
from heapq import heappush as push, heappop as pop

In [3]:
from functools import cache

In [4]:
dict([(0,1)])

{0: 1}

In [5]:
@cache
def all_path(fmap, start, end):
    map = dict(fmap)
    revmap = {v:k for k,v in map.items()}
    dir= {
        "^":(0,-1),
        "v":(0,1,),
        ">":(1,0),
        "<":(-1,0),
    }
    open = [(0, start, '')]
    minscore = float('inf')
    paths = []
    while open:
        score, n, seq = pop(open)
        import time
        time.sleep(0.01)        
        score = len(seq)
        (x,y) = revmap[n]
        if score > minscore:
            return paths
        if n == end:
            paths.append(seq)
            minscore = len(seq)
            continue
        neigh = [(x+dx, y+dy, d) for d,(dx,dy) in dir.items()]
        #print(neigh)
        for (nx,ny, d) in neigh:
            if (nx, ny) in map:
                push(open, (score +1, map[(nx,ny)], seq+d))
    return []
            
            
        

In [6]:
keypad

(((0, 0), '7'),
 ((1, 0), '8'),
 ((2, 0), '9'),
 ((0, 1), '4'),
 ((1, 1), '5'),
 ((2, 1), '6'),
 ((0, 2), '1'),
 ((1, 2), '2'),
 ((2, 2), '3'),
 ((1, 3), '0'),
 ((2, 3), 'A'))

In [19]:
all_path(keypad, '4', 'A')

['>>vv', '>v>v', '>vv>', 'v>>v', 'v>v>']

In [20]:
all_path(rob, 'A', '^')

['<']

In [28]:
from itertools import pairwise

In [118]:
target = '029A'
target = '2'
for source, target in pairwise(iter('A'+target)):
    for a in all_path(keypad, source, target):
        print('\nFinal keypad:', a)
        l1 = shortest(a, rob)
        print('Level 1', l1)
        print('Level 2', shortest(a, rob,2 ))


Final keypad: <^
Level 1 <v<A>^A>A
Level 2 ((<v<AA>^A)A<AA(vAA<^A>A)A)A>^A>A

Final keypad: ^<
Level 1 <Av<A>>^A
Level 2 <Av<A(vAA(<v<A>^A>A,<Av<A>>^A)A>A)A


First: 
`<vA<AA>>^AvAA<^A>A<v<A>>^AvA^A<vA>^A<v<A>^A>AAvA^A<v<A>A>^AAAvA<^A>A`

In [ ]:
<vA<AA>>^AvAA<^A>A<v<A>>^AvA^A<vA>^A<v<A>^A>AAvA^A<v<A>A>^AAAvA<^A>A
v<<A>>^A<A>AvA<^AA>A<vAAA>^A
<A^A>^^AvvvA

In [110]:
@cache
def shortest(seq, using, depth=0):
    #print('looking for depth', depth, 'seq', seq)
    assert not seq.startswith('A')
    assert 'A' not in seq
    acc = ''
    for source, target in pairwise(iter('A'+seq+'A')):
        if source == target:
            acc += 'A'
            continue
        all = all_path(using, source, target)
        if len(all) == 1:
          acc += all[0]
        elif depth == 0:
            if all:
                acc+=all[0]
            #print([len(a) for a in all])
        else:
            if all:
                possibles = [shortest(a, using,depth-1) for a in all]
                
                lens = [len(s) for s in possibles]
                minlen = min(lens)
                remind= [s for s in possibles if len(s) == minlen]
                acc += '(' + ','.join(remind) + ')'
            else:
                print('nothin from', source, 'to', target)
            
        acc +='A'
    return acc
            
    
    

In [66]:
shortest('>^^', rob, 1)

looking for depth 1 seq >^^
looking for depth 0 seq <^
[3, 3]
<v<A>^A>A
looking for depth 0 seq ^<
[3, 3]
<Av<A>>^A


'vAAA>A'

```
< to move the arm from A (its initial position) to 0.
A to push the 0 button.
^A to move the arm to the 2 button and push it.
>^^A to move the arm to the 9 button and push it.
vvvA to move the arm to the A button and push it.
```